# Downloading Your Data from LCO to Google Drive

In this notebook we will outline a process that will allow you to automatically transfer the data for your target from the LCO Archive directly to a folder in your Google Drive. This will allow you to both access and analyze your science data entirely within the cloud!

Let's start by first importing some necessary items..

In [ ]:
import requests
import numpy as np
import datetime
import json

from IPython.display import Image, display
from IPython.core.display import HTML
from astropy.io import fits

# Accessing the LCO Data Archive

The first step is to gain access to the observing portal using our API token..

In [ ]:
token = "PUT_TOKEN_HERE"
obs_portal = 'https://observe.lco.global/api/requestgroups/{}/'
headers = {'Authorization': 'Token {}'.format(token)}

Next, we can request information about the set of observations we requested using our `request_id`.

Set the `request_id` variable below to your ID (you will be given this by a PASEA-ARP instructor).

In [ ]:
request_id = 'REQ_ID_HERE'
url = obs_portal.format(request_id)
r = requests.get(url, headers=headers, timeout=20.0)

print(r)

In the next cell we will run a line of code that checks to see if the requested observations are completed. If it returns "COMPLETED' then you're good to go! 

In [ ]:
print(r.json()['state'])

Each individual observation made of your target (i.e. each exposure or data frame) is given a unique observation ID number. This means that a given request to LCO will have many observation IDs associated with it. In other words, a given `request_id` will have multiple `obs_ids`.

The code below returns a list (`obs_ids`) of all observation IDs linked to your requested data set.

In [ ]:
obs_ids = [rs['id'] for rs in r.json()['requests']]
print(obs_ids)

In the next step, we will login to the LCO Science Archive (which houses all the public science data taken by LCO) to snag our Archive access token.

In the cell below, fill in `username` and `password` dictionary fields with your own username and password for the LCO Archive. 

If you don't have an account at the LCO Data Archive yet, you can create one [here](https://archive.lco.global/). Note: these values are case-sensitive.

In [ ]:
ARCHIVE_API = 'https://archive.lco.global/?REQNUM={}&start=2014-01-01'
ar = requests.post(
    'https://archive-api.lco.global/api-token-auth/',
    data = {
        'username': 'XXX',
        'password': 'XXX'
    }
)
ar_token = ar.json()['token']
archive_headers={'Authorization': 'Token {}'.format(ar_token)}

Let's start with just one observation ID. Choose one from the `obs_ids` list above and paste it into the `id_num` variable below.

In [ ]:
id_num = #YOUR_OBS_ID_NUM_HERE
reduc_lvl = 91

ar = requests.get(f'https://archive-api.lco.global/frames/?REQNUM={id_num}&RLEVEL={reduc_lvl}',headers=archive_headers,)

ar.status_code

Now let's check and see how many files are included in this individual observation..

In [ ]:
print("You have {} data files ".format(ar.json()['count']))

## Exercise / Question

Hmm! There's TWO! Why might that be? 🤔

Hint: 🌈

One fun thing we can do now that we've linked up with some of the data is to visualize the frames within this single set of exposures..

In [ ]:
for req in ar.json()['results'][0:2]:
    get_thumb = requests.get('https://thumbnails.lco.global/{}/'.format(req['id'])).json()
    display(Image(url= get_thumb['url'], width=300, height=300))
    print(req['id'])

## Exercise / Question

Compare and contrast the two frames above.

Can you explain why the two frames look different?

# Downloading the Archive Data into Directories on Google Drive

Now that we've established how to access data in the LCO Archive using notebooks, we need to prep our Google Drive space to save the data.

let's begin by first mounting our Drive directory so that it can be accessed by this notebook.

⚠️ NOTE: ⚠️ When you run the cell below it will launch a pop up window from Google that will ask you to authorize Colab to access your Google Drive space. This step is _necessary_! You should make sure you're authorizing the right Google account before you proceed.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Next, we need to make a directory in our Google Drive space to house all of our LCO data. Let's call this folder "LCO":

In [ ]:
import os
os.mkdir("/content/drive/MyDrive/LCO/")

If you load up your Google Drive in a new browser tab, you should now see the directory there. 

Another way of checking is by clicking on the folder icon in the left-hand side bar, which shows the full directory tree of your Google Drive.

Remember that a given observation request has a number of observation IDs associated with it. We want to download all of the reduced data for this observation request.

So the next step is to use a `for` loop to iterate over all `obs_ids` and download all the reduced data.

NOTE: This next cell may take a while to run, depending on how many exposures are linked with your request. Please be patient! 

In [ ]:
for id in obs_ids:
  reduc_lvl = 91
  myreq = requests.get(f'https://archive-api.lco.global/frames/?REQNUM={id}&RLEVEL={reduc_lvl}',headers=archive_headers,)
  for req in myreq.json()['results'][0:2]:
    !wget -O /content/drive/MyDrive/LCO/"{req['basename']}".fits.fz "{req['url']}"

Now check out your Google Drive "LCO" directory - the data should be all there! 🙌

## Exercise / Question

Your next step before proceeding to the next notebook is to use the methods you learned in the Python workshop to write some code that allows you to access the headers of these data files and determine the filter used by the telescope. Remember, you requested observations of your target in more than one filter!

Then seperate the data files into TWO folders within the LCO directory, each labeled for the respective filter (i.e. "B", etc).

When you're done, proceed to the Data Analysis II notebook. ➡️

In [ ]:
# YOUR CODE HERE